In [1]:
import java.time.LocalDate

data class SeoulCsvData(
    val viorYmd : LocalDate?, // 위반 일자
    val admDispoYmd : LocalDate?, // 처분일자
    val violCn : String?, // 위반 내용
    val sntCobNm: String, // 업종
    val upsoNm: String, // 업소명
    val siteAddrRd: String?, // 도로명 주소
    val siteAddr : String?, // 지번 주소
    val dispoCtn : String, // 처분 유형
    val dispoGigan : Int?, // 처분 기간
    val basLaw : String?, // 법적근거
)

In [2]:
%use dataframe

import java.time.LocalDate
import kotlin.collections.filter
import java.time.format.DateTimeFormatter

val df = DataFrame.readCSV("seoulAdministrativeDispositionStatus.csv", delimiter = ',', charset = charset("EUC-KR"))

val formatter = DateTimeFormatter.ofPattern("yyyyMMdd")

val startDate = LocalDate.of(2020, 1, 1)
val endDate = LocalDate.of(2025, 12, 31)

val dataList: List<SeoulCsvData> = df.rows()
    .map { row ->
        SeoulCsvData(
            sntCobNm = row["업종명"] as String,
            upsoNm = row["업소명"] as String,
            siteAddrRd = row["소재지도로명"] as? String,
            siteAddr = row["소재지지번"] as? String,
            dispoCtn = (row["처분명"] as String).replace("?"," "),
            dispoGigan = (row["처분기간"] as? Number)?.toInt(),
            admDispoYmd = (row["처분일자"] as? Number)?.toString()?.let {
                if (it.length == 8) LocalDate.parse(it, formatter)
                else null
            },
            violCn = row["위반내용"] as? String,
//            viorYmd = (row["위반일자"] as? Number)?.toString(),
            viorYmd = (row["위반일자"] as? Number)?.toString()?.let {
                if (it.length == 8) LocalDate.parse(it, formatter)
                else null
            },
            basLaw = row["법적근거"] as? String
        )
    }
    .filter { data ->
        data.admDispoYmd?.let { it in startDate..endDate } == true
    }
    .filter { data ->
        data.viorYmd?.let { it in startDate..endDate } == true
    }
    .filter {
        it.sntCobNm in listOf("일반음식점", "휴게음식점", "집단급식소식품판매업", "제과점영업", "집단급식소")
    }
    .filter {
        val keywords = listOf("영업소폐쇄")

        keywords.none { keyword ->
            it.dispoCtn.contains(keyword) == true
        }
    }.distinctBy {
        it.upsoNm to it.viorYmd to it.admDispoYmd to it.violCn to it.dispoCtn
    }
    .filter {
        val keywords = listOf(
            "건강진단", "폐업신고", "면적변경", "무단확장", "변경신고", "간판에 신고한 상호를 표시하지 않음", "폐업", "휴업", "재난배상책임", "간판상호", "직권말소", "영업신고증",
            "전부철거", "사업자등록", "잠금", "간판", "외부영업", "운영제한", "면적", "보험", "영업장 외", "영업장외", "철거", "업종", "상호", "시설개수명령","고발", "가격", "시정명령", "미게시",
            "자동반주장치", "영상반주시설", "자동영상반주장치", "무도장", "방음장치",
            "옥외영업", "무단 확장", "무단확장",
            "외 영업", "옥외영업장", "장소", "광고",
            "주류제공", "주류판매", "청소년", "음주", "미성년자", "주류",
            "춤", "노래방기계", "마이크", "도박행위", "유흥접객", "노래", "음향", "유흥주점영업", "사행행위", "자동반주기", "접객행위", "성매매", "꾀어서",
            "멸실",
            "과태료",
            "광고", "키오스크",
            "코로나", "집합금지", "방역수칙", "사적 모임", "사적모임", "이용자출입명부", "출입자 명부", "명부", "21시 이후", "사회적 거리두기", "22시 이후", "4인 이상", "4인이상", "5인이상", "5인 이상", "운영시간", "운영 시간", "거리두기",
            "미수료", "미필", "미이수", "건강검진", "2017년","2018년", "2019년", "2020년", "2021년","2022년", "2023년", "2024년", "2025년",
        )

        keywords.none { keyword ->
            it.violCn?.contains(keyword) == true
        }
    }

In [3]:
val newDf = dataList.toDataFrame()
newDf.writeCSV("filtered-disposition.csv")

newDf

viorYmd,admDispoYmd,violCn,sntCobNm,upsoNm,siteAddrRd,siteAddr,dispoCtn,dispoGigan,basLaw
2024-11-21,2025-05-02,소비(유통)기한 경과제품 판매(1차),일반음식점,자담치킨 가산점,"서울특별시 금천구 가산로 89, 대명U타운 1층 (가산동)",서울특별시 금천구 가산동 153번지 7호 대명U타운,영업정지,null,법 제71조 및 법 제75조
2025-04-08,2025-05-01,이물혼입(실-야키토리동세트),일반음식점,멘지 롯데마트 양평점,"서울특별시 영등포구 선유로 138, 롯데마트 양평점 1층 일부호 (...",서울특별시 영등포구 양평동3가 45번지 롯데마트 양평점,시정명령,null,법 제71조
2025-04-04,2025-04-29,폐기물용기 뚜껑 없음,일반음식점,백두갈비,"서울특별시 송파구 송파대로 111, 205동 123,124호 (문정...",서울특별시 송파구 문정동 618번지 파크하비오,시설개수명령,null,"법 제71조, 법 제74조 및 법 제75조"
2024-10-04,2025-04-22,이물(플라스틱)혼입,일반음식점,피자나라 치킨공주 이문점,"서울특별시 동대문구 이문로 180, 1층 (이문동)",서울특별시 동대문구 이문동 255번지 255호,시정명령,null,"법 제71조, 법 제72조 및 법 제75조"
2025-04-07,2025-04-16,조리장 내 폐기물용기 뚜껑 미설치(1차),일반음식점,요녀석 송파점,"서울특별시 송파구 백제고분로46길 33, 2층 (송파동)",서울특별시 송파구 송파동 145번지 18호,시설개수명령,null,법 제74조
2025-03-05,2025-03-27,종사자 위생모 및 마스크 미착용,일반음식점,금마차식당,"서울특별시 영등포구 영등포로 219, 3층 (영등포동5가)",서울특별시 영등포구 영등포동5가 20번지 62호,과태료부과 8만원,null,법 제101조제2항제1호
2025-03-05,2025-03-27,종사자 위생모 및 마스크 미착용,일반음식점,금마차식당,"서울특별시 영등포구 영등포로 219, 3층 (영등포동5가)",서울특별시 영등포구 영등포동5가 20번지 62호,과태료부과 16만원(영업주+종업원),null,법 제101조제2항제1호
2025-01-16,2025-02-18,식품 등의 취급 기준 위반(식품취급시설인 조리장에 반려동물 출입),일반음식점,요거트월드 장안점,"서울특별시 동대문구 장한로25길 26, 1층 (장안동)",서울특별시 동대문구 장안동 370번지 8호,"과태료부과 200,000원 및 시설개수 명령(소상공인50%감면 및 ...",null,법 제101조제2항제1호
2025-01-16,2025-02-18,동물의 출입 공간을 분리하지 아니 함,일반음식점,요거트월드 장안점,"서울특별시 동대문구 장한로25길 26, 1층 (장안동)",서울특별시 동대문구 장안동 370번지 8호,"과태료부과 200,000원 및 시설개수 명령(소상공인50%감면 및 ...",null,"법 제71조, 법 제72조 및 법 제75조"
2025-01-15,2025-02-17,음식물 내 이물 혼입(1차),일반음식점,금문 24시,"서울특별시 은평구 역말로9길 3, 1층 (대조동)",서울특별시 은평구 대조동 171번지 28호,시정명령,null,"법 제71조, 법 제72조 및 법 제75조"
